In [13]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import random
from scipy.signal import windows,freqz
from scipy import signal, fft
%run Functions.ipynb

In [26]:
results_df = pd.DataFrame(columns = ['length', 'L', 'theta', 'f_stat', 'L_hat', 'theta_hat'])

In [27]:
for length in [100,250,500,1000]:
    results = []
    bandwidth = length**(-3/4)
    K = int(2*bandwidth*length-math.log(length)**2/20)
    wins, eigvals = windows.dpss(M = length, NW = bandwidth*length, Kmax = K, return_ratios=True)
    for M in [3,5,6,10]:
        for a in range(100):
            noise = Noise(length, 'Lomax')
            gridSize = int(length**(1.5)*math.log(length)/math.log(10))
            sigma_max = 20
            freq = 0.035 + random.random()/100
            y = noise.linearProcess([0.75, -0.5, 0, 0])
            signalSample = y

            non_null = [M]
            randAmps = []
            randPhases = []
            while np.gcd.reduce(non_null) != 1:
                non_null = [M]
                for i in range(1,M):
                    randAmp = random.random()
                    if randAmp < 0.2:
                        randAmp = 0
                    else:
                        non_null.append(i)
                    randPhases.append(random.random()*2*math.pi)
                    randAmps.append(randAmp)
                randAmp = random.random() + 0.2
                randPhase = random.random()*2*math.pi   
                randPhases.append(random.random()*2*math.pi)
                randAmps.append(randAmp)

            randAmps = math.sqrt(2)/np.sum(np.square(randAmps))*np.array(randAmps)
            for i in range(1,M+1):
                signalSample += randAmps[i-1]*np.cos(2*i*np.pi*np.arange(length)*(freq)+randPhases[i-1])

            max_fstat, argmax_freq, argmax_mult = get_waveShapeFStat_unknownL(signalSample, gridSize, bandwidth, 1/2-bandwidth, sigma_max, 4)

            results.append((length, M, freq, max_fstat, argmax_mult, argmax_freq))
    results = pd.DataFrame(results, columns = ['length', 'L', 'theta', 'f_stat', 'L_hat', 'theta_hat'])
    results_df = pd.concat([results_df,results])

KeyboardInterrupt: 

In [ ]:
for L in [3,5,6,10]:
    for length in [100, 250,500,1000]:
        accuracy = len(results_df[(abs(results_df['theta_hat']-results_df['theta'])<1/(2*length))
                             &(results_df['L']==L)
                             &(results_df['L_hat']==L)
                             &(results_df['length']==length)].sort_values('f_stat'))
        total = len(results_df[(results_df['L']==L)
                             &(results_df['length']==length)])
        print(f'L = {L}, length = {length}, accuracy = {accuracy/total}')